In [1]:
%cd ../

In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route length
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. I think that emukit handles categorical/discrete inputs using one-hot encodings.

#### Model Inputs

In [6]:
# gridSize = ContinuousParameter('gridSize', min_value=3, max_value=20)
# edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=1, max_value=25)
# maxSpeed = ContinuousParameter('maxSpeed', min_value=1, max_value=25)
# edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=200)
# numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
# accel = ContinuousParameter('accel', 1., 6.)

# parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numberOfLanes, accel])
import experimental_design.config as config
parameter_space = config.get_parameter_space()

#### Initialize Model / Emulator (GP)

In [7]:
#init_X, init_Y = pickle.load(open('experimental_design/init_points/25_init_points.pkl', 'rb'))

design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 20
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
with open(f'experimental_design/init_points/{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)


[[18.         21.29471398 10.84159073 51.710595    1.          3.073451  ]
 [10.         17.71015026 19.38865074 61.40277485  1.          3.61846045]
 [ 3.         22.15671886 15.03720399 48.64065614  3.          1.90655573]
 [17.         15.56086041 12.93259488 42.12967458  1.          1.58924423]
 [13.         13.89272444 13.46640112 43.79690961  1.          1.9259444 ]
 [17.         23.91929617 11.22955358 37.20167828  1.          1.94310127]
 [ 4.         18.46026734  6.31239346 51.20928034  1.          2.49811022]
 [19.         12.01943081 15.5602276  36.19001061  2.          1.99164107]
 [14.         13.11477879 23.96213755 61.00858012  2.          4.06484513]
 [12.         13.39800176 11.35340573 41.94600644  3.          4.24116715]
 [ 7.         21.54224208  6.93008724 54.69510458  3.          3.32276322]
 [10.         10.9497604  14.41182116 34.72894936  2.          2.57731689]
 [15.         13.85158776  7.74701478 35.60437716  3.          3.54300043]
 [ 4.         18.07582123

In [8]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model = GPyModelWrapper(emulator, n_restarts=5)
emukit_model.optimize()
emulator

Optimization restart 1/5, f = -26.105669864682376
Optimization restart 2/5, f = -0.689048801047667
Optimization restart 3/5, f = -26.58451893080403
Optimization restart 4/5, f = -0.689048801047937
Optimization restart 5/5, f = -26.584518930804265


GP_regression.,value,constraints,priors
rbf.variance,0.0324030401455607,+ve,
rbf.lengthscale,16.674498119017887,+ve,
Gaussian_noise.variance,0.00013215882025482766,+ve,


### Optimization

#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 200)

Optimization restart 1/5, f = -26.584518930804265
Optimization restart 2/5, f = -0.6890488010479441
Optimization restart 3/5, f = -20.587381025992233
Optimization restart 4/5, f = -26.584518930796143
Optimization restart 5/5, f = -26.584518930798247
[[20.  8.  5. 70.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -26.03080986028283
Optimization restart 2/5, f = -0.655731849735016
Optimization restart 3/5, f = -0.65573166339437
Optimization restart 4/5, f = -0.6557316633979173
Optimization restart 5/5, f = -26.030809860224757
[[ 3. 25. 27. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -23.918184593131965
Optimization restart 2/5, f = 1.595758927145491
Optimization restart 3/5, f = 1.5957589433699049
Optimization restart 4/5, f = 1.5957589421716332
Optimization restart 5/5, f = -23.918184593153264
[[20.  25.  27.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -22.858570813636497
Optimization restart 2/5, f = 3.127924691805422
Optimization restart 3/5, f = 3.1279247381515525
Optimization restart 4/5, f = 3.1279247381515294
Optimizatio

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -27.04295599995865
Optimization restart 2/5, f = 5.655770070617647
Optimization restart 3/5, f = -27.042955999961045
Optimization restart 4/5, f = 5.655770038435039
Optimization restart 5/5, f = -27.042955999955957
[[ 3.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -28.37583671963636
Optimization restart 2/5, f = 5.398360984191868
Optimization restart 3/5, f = -28.375836719697396
Optimization restart 4/5, f = 5.398360984191875
Optimization restart 5/5, f = 5.398360984191942
[[20.        25.        27.        49.0040198  3.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -29.765093631515278
Optimization restart 2/5, f = 6.902718385022171
Optimization restart 3/5, f = 6.902712150349062
Optimization restart 4/

...

 Retrying in 1 seconds
Optimization restart 1/5, f = -323.5029354797071
Optimization restart 2/5, f = -292.8422345026419
Optimization restart 3/5, f = -292.8422348927123
Optimization restart 4/5, f = -292.842237322158
Optimization restart 5/5, f = -323.5029354805873
[[20.         23.15883539 23.3841165  56.75064627  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -325.30152081149527
Optimization restart 2/5, f = -295.22706400485583
Optimization restart 3/5, f = -295.22706839537864


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -300.1395073998606
Optimization restart 5/5, f = -295.2270686932519
[[ 3.          8.         13.03877166 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -327.42453461521995
Optimization restart 2/5, f = -327.4245563239192
Optimization restart 3/5, f = -296.6037202626911
Optimization restart 4/5, f = -327.4245563241466
Optimization restart 5/5, f = -296.6036270991576
[[13.          8.          5.         42.67782351  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -329.8469726089748
Optimization restart 2/5, f = -329.8469726153718
Optimization restart 3/5, f = -329.84697261608
Optimization restart 4/5, f = -329.846972616181
Optimization restart 5/5, f = -329.8469726158377
[[ 8.         14.76134526 27.         54.13200

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = -334.3922349291758
Optimization restart 5/5, f = -302.1541674914263


In [13]:
#emukit_model = pickle.load(open('experimental_design/models/5_init_50_loop_model_timeLoss.pkl', 'rb'))

### Evaluate

#### Test on 25 point test set

In [14]:
#test_X, test_Y = pickle.load(open('experimental_design/test_points/25_test_points.pkl', 'rb'))


design = RandomDesign(parameter_space)  # initialize with random points
num_test_points = 10
test_X = design.get_samples(num_test_points)
test_Y = user_function_time_loss(test_X)
with open(f'experimental_design/test_points/{num_test_points}_test_points.pkl', "wb") as f:
      pickle.dump((test_X, test_Y), f)


[[15.          9.72705593 10.27058551 31.34390452  3.          4.94063932]
 [19.         13.92852147 20.34637651 58.27993496  2.          1.74447518]
 [10.         22.71135238 12.17992285 36.52041085  3.          4.59715697]
 [10.         24.5788716  23.52069127 57.21099899  3.          3.39226334]
 [ 3.         18.59984289 25.81546725 33.31834658  3.          4.29792502]
 [ 4.         10.88209666 10.85029336 34.09407136  1.          2.27898786]
 [ 9.         21.27893502 14.44927891 52.2173265   2.          3.38242538]
 [10.         19.47084739 22.42361137 35.80552456  2.          3.0043293 ]
 [ 4.         21.79956825 16.31590776 32.95009075  3.          2.29320308]
 [14.         14.04159792 12.45062444 68.97041527  3.          3.61442616]]

User function time loss called with 10 inputs to simulate

Evaluating input: 1 of 10

 Retrying in 1 seconds

Evaluating input: 2 of 10

 Retrying in 1 seconds

Evaluating input: 3 of 10

 Retrying in 1 seconds

Evaluating input: 4 of 10

 Retrying

In [15]:
pred_Y, pred_Y_variance = emukit_model.predict(test_X)
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  0.03933937505364552


In [16]:
pred_Y*100, test_Y*100

(array([[27.84673898],
        [24.8553864 ],
        [32.59098263],
        [47.27557777],
        [49.69832816],
        [21.18947079],
        [29.96053027],
        [49.39875271],
        [47.35729953],
        [19.62674198]]),
 array([[28.45528455],
        [26.87348398],
        [30.2591251 ],
        [50.10535188],
        [43.86666667],
        [30.03773585],
        [27.40442656],
        [49.71330275],
        [51.58139535],
        [19.93456132]]))

### Save Model

In [17]:
num_init_points = 20
num_loop_iters = 200


with open(f'experimental_design/models/{num_init_points}_init_{num_loop_iters}_loop_model_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model, f)

### Analysis

As we can see, optimizing for model_variance causes emukit to alternate between extreme choices of \[gridSize, edgeMaxSpeed, maxSpeed, numberOfLanes, accel\] whilst only varying edgeLength sensibly. This means that we don't explore the parameter space comprehensively - which is undesirable.